# LightGBM Classifier

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
import pandas as pd
dataset = pd.read_csv('churn_modelling.csv')

In [ ]:
dataset.head()

### Checking missing data

In [ ]:
dataset.info()

### Handling categorical variables

CustomerId and Surname columns

In [ ]:
dataset.drop(['CustomerId', 'Surname'], axis = 1, inplace = True)

In [ ]:
dataset.head()

Geography column

In [ ]:
dataset['Geography'].unique()

In [ ]:
geography_dummies = pd.get_dummies(dataset['Geography'], drop_first = True)

In [ ]:
geography_dummies

In [ ]:
dataset = pd.concat([geography_dummies, dataset], axis = 1)

In [ ]:
dataset.head()

In [ ]:
dataset.drop(['Geography'], axis = 1, inplace = True)

In [ ]:
dataset.head()

Gender column

In [ ]:
dataset['Gender'].unique()

In [ ]:
dataset['Gender'] = dataset['Gender'].apply(lambda x: 0 if x == 'Female' else 1)

In [ ]:
dataset.head()

### Creating the Training Set and the Test Set

Getting the inputs and output

In [ ]:
X = dataset.iloc[:, :-1].values

In [ ]:
y = dataset.iloc[:, -1].values

In [ ]:
X

In [ ]:
y

Getting the Training Set and the Test Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Part 2 - Building and training the model

### Building the model

In [ ]:
import lightgbm as lgb
model = lgb.LGBMClassifier()

### Training the model

In [ ]:
model.fit(X_train, y_train)

### Inference

In [ ]:
y_pred = model.predict(X_test)

### Predicting the result of a single observation

**Homework**

Use our model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [ ]:
print(model.predict([[0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

Therefore, our model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "0, 0" in the first two columns. That's because of course the predict method expects the dummy values of the Geography variable.

## Part 3: Evaluating the model

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

### Accuracy

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

### k-Fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = model,
                             X = X,
                             y = y,
                             scoring = 'accuracy',
                             cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [{'num_leaves': [29,30,31,32,33], 'learning_rate': [0.08,0.09,0.1,0.11,0.12], 'n_estimators': [80,90,100,110,120]}]
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search.fit(X, y)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)